# Utilização de LLMs

In [1]:
import logging
import warnings

# Suprimir avisos específicos de FutureWarning e UserWarning
warnings.filterwarnings("ignore", category=FutureWarning, message=".*TRANSFORMERS_CACHE.*")
warnings.filterwarnings("ignore", message=".*resume_download.*deprecated.*", category=FutureWarning)
warnings.filterwarnings("ignore", message=".*use_cache=True.*", category=UserWarning)
warnings.filterwarnings("ignore", message=".*use_reentrant parameter should be passed explicitly.*", category=UserWarning)
warnings.filterwarnings("ignore", message="torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly.")


# Configurar o nível de log para a biblioteca transformers
logging.getLogger("transformers.trainer").setLevel(logging.WARNING)
logging.getLogger("transformers.trainer_utils").setLevel(logging.WARNING)
logging.getLogger("transformers.training_args").setLevel(logging.WARNING)




In [2]:
import torch
import bitsandbytes
import peft
import accelerate
import datasets
import trl
import warnings
import transformers

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("torch version:", torch.__version__)
print("bitsandbytes version:", bitsandbytes.__version__)
print("peft version:", peft.__version__)
print("accelerate version:", accelerate.__version__)
print("datasets version:", datasets.__version__)
print("trl version:", trl.__version__)
print("transformers version:", transformers.__version__)
print(f"Device name: '{torch.cuda.get_device_name()}'")
print("Device:", device)
print(f"Device properties: '{torch.cuda.get_device_properties(torch.cuda.current_device())}'")
print("Suporta bfloat16." if torch.cuda.is_bf16_supported() else "Não suporta bfloat16.")

torch version: 2.3.1
bitsandbytes version: 0.43.1
peft version: 0.11.1
accelerate version: 0.31.0
datasets version: 2.19.2
trl version: 0.9.6
transformers version: 4.44.0
Device name: 'NVIDIA GeForce GTX 1650'
Device: cuda
Device properties: '_CudaDeviceProperties(name='NVIDIA GeForce GTX 1650', major=7, minor=5, total_memory=3903MB, multi_processor_count=14)'
Suporta bfloat16.


In [3]:
import os
from random import randrange

import torch
import numpy as np
import pandas as pd
from huggingface_hub import login
from datasets import load_dataset, Dataset


from trl import SFTConfig, SFTTrainer
from peft import LoraConfig, prepare_model_for_kbit_training, TaskType, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    TrainerCallback,
    set_seed,
    pipeline,
    TrainerCallback,
    TrainerControl,
    TrainerState,
)

In [4]:
class LanguageModel:

    def __init__(self, tokenizer, model, device):
        self.tokenizer = tokenizer
        self.model = model
        self.device = device
        if self.tokenizer.pad_token_id is None:
            self.tokenizer.pad_token_id = self.tokenizer.eos_token_id

    def tokenize(self, messages):
        text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        model_inputs = tokenizer([text], return_tensors="pt").to(self.device)
        return model_inputs

    def generate(self, messages):
        model_inputs = self.tokenize(messages)
        model_inputs['attention_mask'] = model_inputs['attention_mask'].to(model_inputs['input_ids'].device)
        generated_ids = model.generate(
            model_inputs.input_ids,
            max_new_tokens=2000,
            do_sample=True,
            attention_mask=model_inputs['attention_mask'],
            pad_token_id=self.tokenizer.pad_token_id
        )
        generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)]
        return tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [5]:
# if torch.cuda.is_bf16_supported():
#     compute_dtype = torch.bfloat16
#     # attn_implementation = 'flash_attention_2'
#     attn_implementation = 'eager'
# else:
compute_dtype = torch.float16
attn_implementation = 'eager'

print(attn_implementation)
print(compute_dtype)

eager
torch.float16


# Dataset

In [13]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.text import TextLoader

book = TextLoader("data/1984.txt").load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000, 
    chunk_overlap=50, 
    separators=["\n\n", "\n", " ", ""]
)
paragraphs = text_splitter.split_documents(book)

In [14]:
len(paragraphs)

307

# Modelo de Linguagem

In [15]:
from transformers import AutoModelForCausalLM, AutoTokenizer

login(token=os.environ["HUGGINGFACE_TOKEN"])

model_id = "emdemor/question-generator-v2"
commit_hash = None


# A quantização é uma técnica para reduzir o tamanho do modelo e aumentar a eficiência computacional.
# Utilizamos a classe BitsAndBytesConfig para configurar a quantização em 4 bits, o que reduz o uso de memória e acelera o treinamento.
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype="bfloat16",
        bnb_4bit_use_double_quant=True,
)

# Usamos a classe AutoModelForCausalLM para carregar um modelo pré-treinado adequado para modelagem de linguagem causal.
# Parâmetros importantes incluem:
#  - torch_dtype=compute_dtype: Define o tipo de dado para o modelo.
#  - quantization_config=bnb_config: Aplica a configuração de quantização.
#  - device_map="auto": Distribui automaticamente o modelo nos dispositivos disponíveis.
#  - attn_implementation=attn_implementation: Define a implementação da atenção.
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=compute_dtype,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation=attn_implementation,
    revision=commit_hash,
)

# # adapta o modelo para o treinamento em k-bits, otimizando ainda mais o desempenho.
# model = prepare_model_for_kbit_training(model)

def set_tokenizer(model_id):
    tokenizer = AutoTokenizer.from_pretrained(model_id, revision=commit_hash)
    tokenizer.padding_side = 'right'
    return tokenizer
tokenizer = set_tokenizer(model_id)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3.5-mini-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3.5-mini-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [16]:
import json

llm = LanguageModel(tokenizer, model, device="cuda")

def generate_text(context):
    messages = [
        {
            "content": f"{context}",
            "role": "user"
        },
    ]
    
    with torch.cuda.amp.autocast():
        result = llm.generate(messages)
    
    try:
        return json.dumps(json.loads(result), indent=4, ensure_ascii=False)
    except:
        return result

In [24]:
%%time

paragraph = paragraphs[0]

response = generate_text(paragraph.page_content)

CPU times: user 1min 14s, sys: 34.9 ms, total: 1min 14s
Wall time: 1min 14s


In [25]:
import json

json.loads(response)


[{'pergunta': 'Qual é a época do ano mencionada no texto?',
  'resposta': 'Abril.'},
 {'pergunta': 'Qual é a característica do clima que é ressaltada?',
  'resposta': 'Um dia frio e ensolarado.'},
 {'pergunta': 'Qual é a etnia do personagem que se identifica no contexto?',
  'resposta': 'Judeu.'},
 {'pergunta': 'Quem é Winston Smith mencionado no texto?',
  'resposta': 'Um personagem que se esgueirava pelas portas de vidro do Edifício Vitória.'},
 {'pergunta': 'O que Winston Smith consegue fazer com a onda de pó áspero?',
  'resposta': 'Ele não consegue fugir a tempo, o que leva ao seu contato com a onda de pó.'},
 {'pergunta': 'Como é descrito a escada onde Winston subiu?',
  'resposta': 'Ela apresenta um cartaz colorido de uma cara enorme.'},
 {'pergunta': 'Quem é a imagem representada no cartaz da escada?',
  'resposta': 'O rosto de um homem de uns quarenta e cinco anos.'},
 {'pergunta': 'Qual é o cartaz que Winston encontra na escada?',
  'resposta': 'Alegremente representa a figur

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
You are not running the flash-attention implementation, expect numerical differences.


'[\n    {\n        "pergunta": "Qual era a estação do ano descrita no início do trecho?",\n        "resposta": "É um dia frio e ensolarado de abril."\n    },\n    {\n        "pergunta": "Qual era a fonte de luz fornecida de acordo com o trecho?",\n        "resposta": "Os relógio batiam treze horas."\n    },\n    {\n        "pergunta": "O que estava escondido em Winston Smith que tentava se movimentar?",\n        "resposta": "Winston Smith tinha o queixo fincado no peito, numa tentativa de fugir ao vento impiedoso."\n    },\n    {\n        "pergunta": "O que Winston Smith evitou ao se mover do elevador?",\n        "resposta": "Ele não conseguiu evitar que o acompanhasse uma onda de pó áspero."\n    },\n    {\n        "pergunta": "O que Winston Smith encontrava no saguão do apartamento?",\n        "resposta": "O saguão cheirava a repolho cozido e a capacho de trapos."\n    },\n    {\n        "pergunta": "Descrição de um cartaz que Winston Smith encontra no saguão?",\n        "resposta": 

'01\n\nERA UM DIA FRIO E ENSOLARADO DE ABRIL, E OS RELÓGIOS batiam treze horas. Winston Smith, o queixo fincado no peito, numa tentativa de fugir ao vento impiedoso, esgueirou-se rápido pelas portas de vidro do Edifício Vitória; não porém com rapidez suficiente para evitar que o acompanhasse uma onda de pó áspero. \n\nO saguão cheirava a repolho cozido e a capacho de trapos. Na parede do fundo fora pregado um cartaz colorido, grande demais para exibição interna. \n\nRepresentava apenas uma cara enorme, de mais de um metro de largura: o rosto de um homem de uns quarenta e cinco anos, com espesso bigode preto e traços rústicos mas atraentes. Winston encaminhou-se para a escada. Inútil experimentar o elevador. Raramente funcionava, mesmo no tempo das vacas gordas, e agora a eletricidade era desligada durante o dia. Fazia parte da campanha de economia, preparatória da Semana do Ódio. O apartamento ficava no sétimo andar e Winston, que tinha trinta e nove anos e uma variz ulcerada acima do 

In [31]:
%%time





context = """

O mesmo procedimento estabelecido em teoria dos campos clássicos que nos leva a deduzir as equações de Maxwell, também conduz à eletrodinâmica de Podolsky, desde que a Lagrangia envolva derivadas do tensor intensidade de campo. A partir das equações de campo para Podolsky, que apresenta uma constante de acoplamento a associada à massa do fóton, é possível deduzir a equação de estado para a radiação de Podolsky. Essa equação é do tipo P=w(a,T)ε, em que P é pressão do gás fotônico; ε, a sua densidade de energia e w é o parâmetro da equação barotrópica que depende da temperatura T, além da massa do fóton. Usando essa equação de estado na expressão de conservação do tensor energia-momento de fluido perfeito e na equação de Friedmann, é possível resolver a dinâmica cósmica para um universo preenchido pela radiação de Podolsky. Mostramos que a dinâmica é pouco afetada pela presença de fótons massivos, uma vez que 0,282<wPodolsky<wMaxwell=1/3 para qualquer valor de T, ou equivalentente, do tempo cosmológico t. A correção de Podolsky para a lei de Stefan-Boltzmann é obtida para qualquer valor de temperatura, descrevendo potencialmente desde o universo primordial até o universo atual. Essa correção é relevante no intervalo 0≲ξ≲8 para o parâmetro adimensional ξ=βm. A máxima influência da massa do fóton acontece em ξref=2,899. Fora do intervalo referido intervalo de ξ, a dinâmica cosmológica de Podolsky tende à de Maxwell: nos limites de universo primordial (ξ≪1) e universo atual/futuro (ξ≫1), wPodolsky → wMaxwell e o fator de escala de Podoslsky vai com √t, de maneira consistente com um gás de fótons não massivos.

"""


[
    {
        "pergunta": "O que pode ser deduzido a partir do princípio teórico que leva às equações de Maxwell?",
        "resposta": "A partir do mesmo procedimento estabelecido em teoria dos campos clássicos, é possível deduzir as equações de eletrodinâmica de Podolsky."
    },
    {
        "pergunta": "Qual é a consequência das equações de campo de Podolsky?",
        "resposta": "A partir das equações de campo de Podolsky, é possível deduzir a equação de estado para a radiação de Podolsky."
    },
    {
        "pergunta": "Quais são os componentes da equação de estado de Podolsky?",
        "resposta": "A equação de estado é do tipo P=w(a,T)ε, onde P é a pressão do gás fotônico, ε é a densidade de energia do gás fotônico, e w é o parâmetro da equação barotrópica que depende da temperatura T e da massa do fóton."
    },
    {
        "pergunta": "O que a equação de estado de Podolsky permite resolver em relação à dinâmica cósmica?",
        "resposta": "Usando a equação de est

In [30]:
try:
    print(json.dumps(json.loads(result), indent=2, ensure_ascii=False))
except:
    print(result)

[
  {
    "pergunta": "Qual é a necessidade para se adentrar nos estudos cosmológicos?",
    "resposta": "É necessário entender os desafios que tal jornada propicia."
  },
  {
    "pergunta": "Como é caracterizada a cosmologia em relação à natureza?",
    "resposta": "A cosmologia é uma ciência que aborda aspectos da natureza do universo."
  },
  {
    "pergunta": "Quais são os desafios enfrentados na cosmologia?",
    "resposta": "Destaca-se um enorme abismo entre a cosmologia e as demais ciências no que tange a escalas onde os fenômenos físicos ocorrem."
  },
  {
    "pergunta": "Quais são alguns dos fenômenos que a cosmologia estuda?",
    "resposta": "Galáxias, aglomerados galácticos e todos objetos tais como esses."
  },
  {
    "pergunta": "Quais são as dimensões dos objetos estudados na cosmologia?",
    "resposta": "Esses objetos apresentam dimensões tão superiores as escalas com que estamos habituados."
  },
  {
    "pergunta": "Como deve ser realizada a estudo dos objetos cos

[{'pergunta': 'Quem foi flagrado furtando um celular?',
  'resposta': 'Um idoso.'},
 {'pergunta': 'Onde o furto ocorreu?',
  'resposta': 'Dentro de um estúdio de tatuagem na Rua prefeito Chagas, no centro de Poços de Caldas.'},
 {'pergunta': 'Quando o furto aconteceu?',
  'resposta': 'Na tarde desta terça-feira, dia 20 de agosto.'},
 {'pergunta': 'Qual era a razão pela qual o idoso frequentava o estúdio regularmente?',
  'resposta': 'O idoso costuma ir ao estúdio pedir ajuda para comprar leite para o neto.'},
 {'pergunta': 'Além de pedir ajuda para leite, o que mais o idoso de aproximadamente 70 anos fazia?',
  'resposta': 'Afirmava que pediam ajuda financeira aos comerciantes vizinhos.'},
 {'pergunta': 'O que os comerciantes faziam no momento que a ajuda foi solicitada?',
  'resposta': 'Eles ajudavam sempre que podiam.'},
 {'pergunta': 'O que o tatuador estava fazendo quando o celular foi desaparecido?',
  'resposta': 'O tatuador estava trabalhando e precisou usar o celular para entra

In [12]:
%%time
llm = LanguageModel(tokenizer, model, device="cuda")

context = """
Dois jovens de 19 e 23 anos foram presos na noite desta sexta-feira,16, após disparos de arma de fogo próximo a um bar em Poços de Caldas.

Segundo a Polícia Militar, durante uma operação na Avenida Marechal Castelo Branco, no bairro Jardim São Paulo, os policiais ouviram dois disparos de arma de fogo.

Clique aqui e participe do grupo de notícias da Onda Poços no WhatsApp

Em seguida foi recebida uma denúncia, via 190, relatando que o jovem de 19 anos trajando camiseta do flamengo havia efetuado dois disparos de arma de fogo próximo a um bar e que ele havia repassado a arma a um segundo indivíduo que trajava blusa com capuz preta e bermuda de cor preta, que após pegar arma saiu correndo em direção à Rua Engenheiro Ubirajara Machado de Moraes.

 

Ainda segundo a PM, o jovem de 19 anos já conhecido no meio policial por posse ilegal de arma e tráfico de drogas, foi localizado e abordado.

 

Uma outra equipe da Polícia Militar recebeu denúncias, via 190, de que o segundo suspeito que havia fugido do local com a arma de fogo, a deixou em uma lixeira na Rua Abrieiro. A arma foi apreendida.

 

Ainda segundo a PM, após capturar o primeiro suspeito, a equipe continuou o rastreamento para localizar o segundo envolvido. O jovem foi encontrado ao final da Rua Engenheiro Ubirajara Machado de Moraes com as mesmas vestes repassadas pela denúncia.

 
A arma de fogo se tratava de um revólver calibre 38 marca Taurus, que foi apreendida. Ainda segundo a PM, no local e nas adjacências não foram localizadas vítimas e não foi detectado o local exato em que o projétil atingiu.

A dupla foi presa, encaminhada à UPA e em seguida levada para a Delegacia de Polícia Civil.
"""


messages = [
    {
        "content": "Você é um assistente especializado em interpretação de texto",
        "role": "system"
    },
    {
        "content": f"Gere uma pergunta para o seguinte contexto:\n```\n{context}\n```\nPergunta:",
        "role": "user"
    },
]

with torch.cuda.amp.autocast():
    # model.generate(**tokenizer("test", return_tensors="pt").to("cuda"))
    question = llm.generate(messages)
    print("question =", question)

completion = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": f"Dado o seguinte contexto ```{context}``` responda a pergunta: `{question}`."}
  ]
)


print("response =", completion.choices[0].message.content)

question = Quem saiu correndo do bar após ser preso?
response = O jovem de 19 anos, que havia disparado a arma e repassado a arma a um segundo indivíduo, foi quem saiu correndo em direção à Rua Engenheiro Ubirajara Machado de Moraes após os disparos. No entanto, ele foi localizado e abordado pela Polícia Militar. Portanto, ninguém "saiu correndo do bar após ser preso", já que após serem abordados, os dois jovens foram presos.
CPU times: user 10.2 s, sys: 6.15 s, total: 16.3 s
Wall time: 18.4 s


In [11]:
from openai import OpenAI

client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])


In [120]:
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate

from langchain_core.messages.system import SystemMessage
from langchain_core.messages.human import HumanMessage

import re


import json


def get_list(texto):
    resultado = re.search(r'\[([\s\S]*)\]', texto)
    if resultado:
        conteudo = resultado.group(1)
        return json.loads(f"[{conteudo}]")
    else:
        return "Nenhum conteúdo encontrado entre os colchetes."



def get_questions(context):
    llm = ChatOpenAI(model="gpt-4o-mini")
    
    prompt_text = f"""
    Dado o seguinte contexto:
    ```
    {context}
    ```
    Escreva um conjunto de perguntas e respostas sobre o contexto.
    As perguntas devem ser tais que, concatenando as respostas, seja possível recuperar o conteúdo completo do contexto.
    As perguntas deve ser autocontidas; ou seja, um leitor deve entendê-la sem neessariamente conhecer o contexto

    
    
    # Formato de saída:
    O resultado deve ser parseavel em json:

    ```
    [
     {{"pergunta": "<<pergunta 1>>", "pergunta": "<<resposta 1>>"}},
     {{"pergunta": "<<pergunta 2>>", "pergunta": "<<resposta 2>>"}},
     ...
    ]
    ```
    """
    
    chat_prompt = ChatPromptTemplate.from_messages([
        SystemMessage(content="You are a helpful assistant."),
        HumanMessage(content=prompt_text)
    ])
    
    chain = chat_prompt | llm
    
    with get_openai_callback() as cb:
        response = chain.invoke(dict(context=context))
        preco_real = 5.7 * (0.15 * cb.prompt_tokens+ 0.6*cb.completion_tokens) / 1_000_000
        print(f"preco = {preco_real}")
        
    return get_list(response.content), preco_real


In [22]:
context = """
A ciência da cosmologia sofreu profundas alterações nas últimas décadas. Até então,
a ausência de dados era o maior obstáculo para o estudo dos fenômenos cósmicos. Muitas
das vezes, eram os aspectos filosóficos que ditavam as características de modelos elaborados.
Como exemplo, temos o acréscimo da constante cosmológica por Einstein em suas equações
da Relatividade Geral com o objetivo de descrição de um universo estático, em equilíbrio
(instável)(EINSTEIN, 1918). Em contrapartida, Alexander Friedmann abandonou a premissa de um universo estático e encontrou a descrição analítica de um universo dinâmico
perante a solução das equações de campo da Relatividade Geral (FRIEDMANN, 1922;
FRIEDMANN, 1924). Tal modelo descreve a evolução de um universo proveniente de uma
singularidade, que posteriormente seria cunhado por Lemaître como o átomo primordial.
Uma solução dinâmica para a evolução do universo foi encontrada também por Georges
Lemaître em 1927 (LEMAÎTRE, 1927).


"""

prompt = f"""
Dado o seguinte contexto:
```
{context}
```
Escreva um conjunto de perguntas e respostas sobre o contexto. As perguntas devem ser tais que, concatenando as respostas, seja possível recuperar o conteúdo completo do contexto

# Formato de saída:

[
 {{"pergunta_1": "resposta_1"}},
 {{"pergunta_2": "resposta_2"}},
 {{"pergunta_3": "resposta_3"}},
 ...
]
"""



completion = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
  ]
)


print("response =", completion.choices[0].message.content)

response = [
 {"Qual foi o maior obstáculo para o estudo dos fenômenos cósmicos até algumas décadas atrás?": "A ausência de dados."},
 {"O que ditava as características dos modelos elaborados antes das profundas alterações na cosmologia?": "Aspectos filosóficos."},
 {"Qual foi o objetivo de Einstein ao adicionar a constante cosmológica em suas equações da Relatividade Geral?": "Descrever um universo estático, em equilíbrio (instável)."},
 {"Quem abandonou a premissa de um universo estático e qual foi sua contribuição?": "Alexander Friedmann, que encontrou a descrição analítica de um universo dinâmico."},
 {"O que as soluções das equações de campo da Relatividade Geral de Friedmann descrevem?": "A evolução de um universo proveniente de uma singularidade."},
 {"Quem cunhou o termo 'átomo primordial' para descrever o modelo dinâmico do universo de Friedmann?": "Georges Lemaître."},
 {"Em que ano Georges Lemaître encontrou uma solução dinâmica para a evolução do universo?": "1927."}
]


In [121]:
 
context = """
A ciência da cosmologia sofreu profundas alterações nas últimas décadas. Até então,
a ausência de dados era o maior obstáculo para o estudo dos fenômenos cósmicos. Muitas
das vezes, eram os aspectos filosóficos que ditavam as características de modelos elaborados.
Como exemplo, temos o acréscimo da constante cosmológica por Einstein em suas equações
da Relatividade Geral com o objetivo de descrição de um universo estático, em equilíbrio
(instável)(EINSTEIN, 1918). Em contrapartida, Alexander Friedmann abandonou a premissa de um universo estático e encontrou a descrição analítica de um universo dinâmico
perante a solução das equações de campo da Relatividade Geral (FRIEDMANN, 1922;
FRIEDMANN, 1924). Tal modelo descreve a evolução de um universo proveniente de uma
singularidade, que posteriormente seria cunhado por Lemaître como o átomo primordial.
Uma solução dinâmica para a evolução do universo foi encontrada também por Georges
Lemaître em 1927 (LEMAÎTRE, 1927).
"""

response, preco_real = get_questions(context)

response

preco = 0.0015313049999999997


[{'pergunta': 'Quais foram as principais mudanças na ciência da cosmologia nas últimas décadas?',
  'resposta': 'A ciência da cosmologia sofreu profundas alterações, com a superação da ausência de dados que antes era o maior obstáculo para o estudo dos fenômenos cósmicos.'},
 {'pergunta': 'Como os aspectos filosóficos influenciaram os modelos cosmológicos anteriormente?',
  'resposta': 'Os aspectos filosóficos muitas vezes ditavam as características de modelos elaborados, como no caso do acréscimo da constante cosmológica por Einstein para descrever um universo estático.'},
 {'pergunta': 'Qual foi a contribuição de Einstein para a cosmologia?',
  'resposta': 'Einstein acrescentou a constante cosmológica em suas equações da Relatividade Geral com o objetivo de descrever um universo estático em equilíbrio.'},
 {'pergunta': 'Quem foi Alexander Friedmann e qual foi sua contribuição para a cosmologia?',
  'resposta': 'Alexander Friedmann abandonou a ideia de um universo estático e encontrou

In [75]:


columns = ['id', 'context', 'question','response']
train_dataset = dataset["train"].shuffle(42).select(range(5000)).select_columns(columns)
test_dataset = dataset["validation"].shuffle(42).select(range(100)).select_columns(columns)

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

Como nas outras línguas românicas ocidentais, a principal expressão plural é o sufixo -s, que pode criar alternâncias morfológicas semelhantes às encontradas na inflexão de gênero, embora mais raramente. O mais importante é o acréscimo, antes de certos grupos de consoantes, a um fenómeno fonético que não afeta as formas femininas: el pols / els polsos (&quot;o pulso&quot; / &quot;os pulsos&quot;) vs. la pols / les pols ( &quot;a poeira&quot; / &quot;as poeiras&quot;).

preco = 0.0010225800000000001

[{'pergunta': 'Qual é a principal expressão plural nas línguas românicas ocidentais?', 'resposta': 'O sufixo -s.'}, {'pergunta': 'O que o sufixo -s pode criar nas línguas românicas ocidentais?', 'resposta': 'Alternâncias morfológicas semelhantes às encontradas na inflexão de gênero.'}, {'pergunta': 'O fenômeno fonético que afeta a formação do plural nas línguas românicas ocidentais tem impacto nas formas femininas?', 'resposta': 'Não, ele não afeta as formas femininas.'}, {'pergunta': 'Dê u

In [77]:
for r in test_dataset:
    pass

In [25]:
from langchain.callbacks import get_openai_callback
from langchain_openai import ChatOpenAI

chatmodel = ChatOpenAI(model="gpt-4o-mini", temperature=0, max_tokens=500)


Tokens Used: 17
	Prompt Tokens: 8
	Completion Tokens: 9
Successful Requests: 1
Total Cost (USD): $0.0


Hello! How can I assist you today?
